In [1]:
# Clean ADR1 Vector Plot Code
# Load required libraries
library(tidyverse)
library(ggplot2)
library(ggrepel)
# Load and prepare data
Res_merged <- readRDS("generated_data/Res_merged.rds")

# Change MEcyan to MEruby and fix source names
Res_merged <- Res_merged %>%
  mutate(across(c(module, Module_n), ~gsub("MEcyan", "MEruby", .)))
levels(Res_merged$source) <- gsub("/", "-", levels(Res_merged$source))

# Calculate median log2FC by source and module
df_median <- Res_merged %>%
  group_by(source, module) %>%
  summarise(l2fc_median = median(log2FC, na.rm = TRUE), .groups = 'drop') %>%
  spread(module, l2fc_median)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Add classifications
df_plot <- df_median %>%
  mutate(
    data_source = case_when(
      str_detect(source, "^AvrRpt2-|^AvrRps4-|^DM1-DM2d|^DM6-DM7|^DM10-DM11|_adr1s$") ~ "Our data",
      TRUE ~ "Published data"
    ),
    immune_type = case_when(
      str_detect(source, "Rpt|Rpm|Rps") ~ "infection",
      TRUE ~ "autoimmune"
    ),
    shape_type = if_else(immune_type == "autoimmune", "circle", "triangle")
  )

# Create ADR1 vector pairs manually
adr1_vectors <- tibble(
  wt_name = c("DM1-DM2d", "DM6-DM7", "Col-0"),
  ko_name = c("DM1-DM2d_adr1s", "DM6-DM7_adr1s", "Col-0_adr1s")
)

# Get coordinates for vector plot
vector_data <- map_dfr(1:nrow(adr1_vectors), function(i) {
  wt_data <- df_plot %>% filter(source == adr1_vectors$wt_name[i])
  ko_data <- df_plot %>% filter(source == adr1_vectors$ko_name[i])
  
  # Handle Col-0 as origin (0,0) since it's not real data
  if(adr1_vectors$wt_name[i] == "Col-0") {
    tibble(
      wt_name = "Col-0",
      ko_name = "Col-0_adr1s",
      x_start = 0,
      y_start = 0,
      x_end = ko_data$MEblue,
      y_end = ko_data$MEruby,
      data_source = "Our data"
    )
  } else {
    tibble(
      wt_name = adr1_vectors$wt_name[i],
      ko_name = adr1_vectors$ko_name[i],
      x_start = wt_data$MEblue,
      y_start = wt_data$MEruby,
      x_end = ko_data$MEblue,
      y_end = ko_data$MEruby,
      data_source = wt_data$data_source
    )
  }
})

# Define colors and shapes
colors <- c("Our data" = "#D2691E", "Published data" = "#4682B4")
shapes <- c("circle" = 16, "triangle" = 17)

In [3]:
width_ <- 5
height_ <- 5
options(repr.plot.width = width_, repr.plot.height = height_, repr.plot.res = 200)

# Create the plot
p_adr1 <- ggplot() +
  # Grid lines
  geom_vline(xintercept = 0, lty = 3, color = "gray50") + 
  geom_hline(yintercept = 0, lty = 3, color = "gray50") +
  
  # Background points (all data, faded)
  geom_point(data = df_plot, 
             aes(x = MEblue, y = MEruby, 
                 color = data_source, 
                 shape = shape_type),
             alpha = 0.5, size = 2) +
  
  # Highlighted wild-type points
  geom_point(data = df_plot %>% filter(source %in% c("DM1-DM2d", "DM6-DM7")),
             aes(x = MEblue, y = MEruby, 
                 color = data_source, 
                 shape = shape_type),
             size = 3) +
  
  # Col-0 reference point at origin (multiple debugging approaches)
  # Method 1: Using data frame (most reliable)
  geom_point(data = data.frame(x = 0, y = 0), 
             aes(x = x, y = y), 
             size = 3, shape = 16, color = "#9CCF2D", stroke = 1) +
  
  # # Method 2: Add a border to make it more visible
  # geom_point(data = data.frame(x = 0, y = 0), 
  #            aes(x = x, y = y), 
  #            size = 4, shape = 21, fill = "#9CCF2D", color = "black", stroke = 1) +
  
  # Vector arrows
  geom_segment(data = vector_data,
               aes(x = x_start, y = y_start, 
                   xend = x_end, yend = y_end),
               arrow = arrow(length = unit(0.02, "npc"), type = "closed"),
               color = "black", linewidth = 0.5) +
  
  # Labels (with origin point debugging)
  geom_text(data = vector_data %>% filter(wt_name != "Col-0"),
            aes(x = x_start, y = y_start, label = wt_name),
            size = 3, hjust = 1.1, vjust = -0.3) +
  
  # Make sure Col-0 label is visible
  geom_text(data = data.frame(x = 0, y = 0, label = "Col-0"),
            aes(x = x, y = y, label = label),
            size = 3, hjust = 1.1, vjust = -0.3, color = "black") +
  
  # Scales and theme
  scale_color_manual(values = colors, name = "Data Source") +
  scale_shape_manual(values = shapes, name = "Condition Type",
                     labels = c("circle" = "Autoimmune", "triangle" = "Infection")) +
  
  labs(
    x = "growth (MEblue log2FC median)",
    y = "immunity (MEruby log2FC median)"
  ) +
  
  xlim(-3.5, 1) +
  ylim(-0.8, 3) +
  
  theme_bw() +theme_Publication() +
  theme(
    legend.position = "bottom",
    axis.text = element_text(size = 8),
    axis.title = element_text(size = 8),
    text = element_text(size = 8, family = "Arial")
  ) + 
  
  guides(
    color = guide_legend(override.aes = list(alpha = 1, size = 3), order = 1),
    shape = guide_legend(override.aes = list(alpha = 1, size = 3), order = 2)
  )

p_adr1 
ggsave("plots/fig_adr1_vector_simplified3.pdf", p_adr1, h = height_, w = width_)

ERROR: Error in theme_Publication(): could not find function "theme_Publication"


In [ ]:
custom_colors <- c(
  # From your provided color set
  "DM6-DM7" = "#F8766D", #F8766D
  "DM1-DM2d" =  "#BC7FB7", 
  "DM10-DM11" = "#A58AFF",
  "Col-0" = "#00C094",    # Green color for Col-0
  "Col-0_adr1s" = "#2A6EDB",
  "Tuescha-9" = "#07a822", 
  "Cdm-0" = "#00A9FF",
  "control" = "#72A063",
  "Mock-6h" = "#94B6CD", 
  "Mock-12h" = "#6D9AB9", 
  "Mock-20h" = "#4579A1",
  "DM6-DM7_adr1s" = "#568FE8",
  "DM1-DM2d_adr1s" = "#1132A9",
  
  # With the specific adaptations you requested
  "AvrRps4" = "#a884ba",  # All AvrRps4 timepoints
  "AvrRpt2" = "#f09f67",  # All AvrRpt2 timepoints
  "ssi2" = "#f4511e",     # All ssi2 variants
  
  # Additional colors for any other categories in your data
  "acd6_9am" = "#EDC5BF",
  "bak1serk4" = "#d50000",
  "hos15" = "#ff6e40",
  "ka120" = "#ff3d00",
  "Est_R4_4h" = "#dd2c00",
  "dm2h_NIL" = "#bf360c",
  "sulki" = "#ff8a65",
  "Saile_Rpm1_30m" = "#00A9FF",
  "Saile_Rpm1_4h" = "#00A9FF",
  "Saile_Rpm1_8h" = "#00A9FF",
  "Saile_Rpt2_30m" = "#00A9FF",
  "Saile_Rpt2_4h" = "#00A9FF",
  "Saile_Rpt2_8h" = "#00A9FF",
  "Saile_Rps4_30m" = "#00A9FF",
  "Saile_Rps4_4h" = "#00A9FF",
  "Saile_Rps4_8h" = "#00A9FF"
)

In [ ]:
# Add fine immune type classification to df_plot
df_plot <- df_plot %>%
  mutate(
    fine_immune_type = case_when(
      # Group AvrRps4 timepoints
      str_detect(source, "^AvrRps4-") ~ "AvrRps4",
      # Group AvrRpt2 timepoints  
      str_detect(source, "^AvrRpt2-") ~ "AvrRpt2",
      # Group ssi2 variants
      str_detect(source, "^ssi2-") ~ "ssi2",
      # Group Saile samples
      str_detect(source, "^Saile_") ~ "Saile",
      # Keep individual names for others
      TRUE ~ as.character(source)
    )
  )

# Update custom_colors to include grouped categories
custom_colors_grouped <- c(
  custom_colors,
  "AvrRps4" = "#a884ba",
  "AvrRpt2" = "#f09f67", 
  "ssi2" = "#f4511e",
  "Saile" = "#00A9FF"
)


# Set plot dimensions for autoimmune plot
width_auto <- 3.6
height_auto <- 5.3

# Create autoimmune plot
df_autoimmune <- df_plot %>% 
  filter(immune_type == 'autoimmune')

p_autoimmune <- ggplot(df_autoimmune, aes(y = MEruby, x = MEblue)) +
  # Reference lines
  geom_vline(xintercept = 0, linetype = "dotted", color = "grey60") + 
  geom_hline(yintercept = 0, linetype = "dotted", color = "grey60") +
  
  # Main points colored by fine immune type
  geom_point(aes(color = fine_immune_type), size = 3, alpha = 0.8) +
  
  # Col-0 reference point at origin
  geom_point(x = 0, y = 0, size = 4, shape = 16, color = "#9ccf2d", alpha = 0.9) +
  
  # Ideal plant marker
  geom_point(x = 2.5, y = 0.5, size = 5, shape = 8, color = "black", alpha = 1) +
  
  # Labels
  geom_text_repel(aes(label = source), size = 2, max.overlaps = Inf,
                  box.padding = 0.3, point.padding = 0.2) +
  
  # Scales and styling
  scale_color_manual(values = custom_colors_grouped, name = "Sample Type") +
  labs(x = "Growth (MEblue log2FC median)",
       y = "Immunity (MEruby log2FC median)") +
  coord_cartesian(xlim = c(-3.5, 1), ylim = c(-0.8, 3)) +
  
  # Theme
  theme_Publication() +
  theme(
    legend.position = "bottom",
    legend.title = element_text(size = 10),
    legend.text = element_text(size = 8)
  ) +
  
  # Legend formatting
  guides(color = guide_legend(override.aes = list(size = 3), ncol = 2))

# Display autoimmune plot
print(p_autoimmune)

# Save autoimmune plot
ggsave("plots/fig_autoimmune_labeled.pdf", p_autoimmune, 
       width = width_auto, height = height_auto, units = "in")
ggsave("plots/fig_autoimmune_labeled.png", p_autoimmune, 
       width = width_auto, height = height_auto, units = "in", dpi = 300)

# Set plot dimensions for infection plot  
width_inf <- 3.6
height_inf <- 5.1

# Create infection plot
df_infection <- df_plot %>% 
  filter(immune_type == 'infection')

p_infection <- ggplot(df_infection, aes(y = MEruby, x = MEblue)) +
  # Reference lines
  geom_vline(xintercept = 0, linetype = "dotted", color = "grey60") + 
  geom_hline(yintercept = 0, linetype = "dotted", color = "grey60") +
  
  # Main points colored by fine immune type
  geom_point(aes(color = fine_immune_type), size = 3, alpha = 0.8) +
  
  # Col-0 reference point at origin
  geom_point(x = 0, y = 0, size = 4, shape = 16, color = "#9ccf2d", alpha = 0.9) +
  
  # Ideal plant marker
  geom_point(x = 2.5, y = 0.5, size = 5, shape = 8, color = "black", alpha = 1) +
  
  # Labels
  geom_text_repel(aes(label = source), size = 2, max.overlaps = Inf,
                  box.padding = 0.3, point.padding = 0.2) +
  
  # Scales and styling
  scale_color_manual(values = custom_colors_grouped, name = "Sample Type") +
  labs(x = "Growth (MEblue log2FC median)",
       y = "Immunity (MEruby log2FC median)") +
  coord_cartesian(xlim = c(-3.5, 1), ylim = c(-0.8, 3)) +
  
  # Theme
  theme_Publication() +
  theme(
    legend.position = "bottom",
    legend.title = element_text(size = 10),
    legend.text = element_text(size = 8)
  ) +
  
  # Legend formatting
  guides(color = guide_legend(override.aes = list(size = 3), ncol = 2))

# Display infection plot
print(p_infection)

# Save infection plot
ggsave("plots/fig_infection_labeled.pdf", p_infection, 
       width = width_inf, height = height_inf, units = "in")
ggsave("plots/fig_infection_labeled.png", p_infection, 
       width = width_inf, height = height_inf, units = "in", dpi = 300)

# Print summary
cat("Plots created and saved:\n")
cat("- Autoimmune plot:", nrow(df_autoimmune), "samples\n")
cat("- Infection plot:", nrow(df_infection), "samples\n")
cat("- Files saved to plots/ directory\n")

In [ ]:
# Load required libraries
library(tidyverse)
library(ggplot2)
library(ggrepel)

# Load and prepare data
Res_merged <- readRDS("generated_data/Res_merged.rds")

# Change MEcyan to MEruby and fix source names
Res_merged <- Res_merged %>%
  mutate(across(c(module, Module_n), ~gsub("MEcyan", "MEruby", .)))
levels(Res_merged$source) <- gsub("/", "-", levels(Res_merged$source))

# Calculate median log2FC by source and module
df_median <- Res_merged %>%
  group_by(source, module) %>%
  summarise(l2fc_median = median(log2FC, na.rm = TRUE), .groups = 'drop') %>%
  spread(module, l2fc_median)

# Add classifications to df_plot
df_plot <- df_median %>%
  mutate(
    # Data source classification
    data_source = case_when(
      str_detect(source, "^AvrRpt2-|^AvrRps4-|^DM1-DM2d|^DM6-DM7|^DM10-DM11|_adr1s$") ~ "Our data",
      TRUE ~ "Published data"
    ),
    
    # Immune type classification
    immune_type = case_when(
      str_detect(source, "Rpt|Rpm|Rps") ~ "infection",
      TRUE ~ "autoimmune"
    ),
    
    # Fine immune type classification for coloring
    fine_immune_type = case_when(
      # Group AvrRps4 timepoints
      str_detect(source, "^AvrRps4-") ~ "AvrRps4",
      # Group AvrRpt2 timepoints  
      str_detect(source, "^AvrRpt2-") ~ "AvrRpt2",
      # Group ssi2 variants
      str_detect(source, "^ssi2-") ~ "ssi2",
      # Group Saile samples
      str_detect(source, "^Saile_") ~ "Saile",
      # Keep individual names for others
      TRUE ~ as.character(source)
    ),
    
    # Shape based on data source
    shape_type = data_source
  )

# Define custom colors
custom_colors_grouped <- c(
  # Your experimental data colors
  "DM6-DM7" = "#F8766D",
  "DM1-DM2d" = "#BC7FB7", 
  "DM10-DM11" = "#A58AFF",
  "DM6-DM7_adr1s" = "#568FE8",
  "DM1-DM2d_adr1s" = "#1132A9",
  "Col-0_adr1s" = "#2A6EDB",
  
  # Grouped experimental data
  "AvrRps4" = "#a884ba",
  "AvrRpt2" = "#f09f67",
  
  # Published data colors
  "ssi2" = "#f4511e",
  "acd6_9am" = "#EDC5BF",
  "bak1serk4" = "#d50000",
  "hos15" = "#ff6e40",
  "ka120" = "#ff3d00",
  "Est_R4_4h" = "#dd2c00",
  "dm2h_NIL" = "#bf360c",
  "sulki" = "#ff8a65",
  "Saile" = "#00A9FF"
)

# Define shapes: circles for our data, triangles for published data
shape_mapping <- c(
  "Our data" = 16,        # solid circle
  "Published data" = 17   # solid triangle
)

# Create directory for plots
dir.create("plots", showWarnings = FALSE)

In [ ]:
# === AUTOIMMUNE PLOT ===
width_auto <- 3.6
height_auto <- 5.3

df_autoimmune <- df_plot %>% 
  filter(immune_type == 'autoimmune')

p_autoimmune <- ggplot(df_autoimmune, aes(y = MEruby, x = MEblue)) +
  # Reference lines
  geom_vline(xintercept = 0, linetype = "dotted", color = "grey60") + 
  geom_hline(yintercept = 0, linetype = "dotted", color = "grey60") +
  
  # Main points with shape distinction
  geom_point(aes(color = fine_immune_type, shape = shape_type), 
             size = 3, alpha = 0.8) +
  
  # Col-0 reference point at origin (using data frame method)
  geom_point(data = data.frame(x = 0, y = 0), 
             aes(x = x, y = y), 
             size = 4, shape = 16, color = "#9ccf2d", alpha = 0.9) +
  
  # Ideal plant marker
  geom_point(x = 2.5, y = 0.5, size = 5, shape = 8, color = "black", alpha = 1) +
  
  # Labels
  geom_text_repel(aes(label = source), size = 2, max.overlaps = Inf,
                  box.padding = 0.3, point.padding = 0.2) +
  
  # Col-0 label
  geom_text(data = data.frame(x = 0, y = 0, label = "Col-0"),
            aes(x = x, y = y, label = label),
            size = 2.5, hjust = 1.1, vjust = -0.3, color = "black") +
  
  # Scales and styling
  scale_color_manual(values = custom_colors_grouped, name = "Sample Type") +
  scale_shape_manual(values = shape_mapping, name = "Data Source") +
  
  labs(x = "Growth (MEblue log2FC median)",
       y = "Immunity (MEruby log2FC median)",
       title = "Autoimmune Conditions") +
  
  coord_cartesian(xlim = c(-3.5, 1), ylim = c(-0.8, 3)) +
  
  # Theme
  theme_bw() +
  theme(
    legend.position = "bottom",
    legend.title = element_text(size = 10),
    legend.text = element_text(size = 8),
    axis.text = element_text(size = 8),
    axis.title = element_text(size = 10),
    plot.title = element_text(size = 12, hjust = 0.5)
  ) +
  
  # Legend formatting
  guides(
    color = guide_legend(override.aes = list(size = 3), ncol = 3, order = 1),
    shape = guide_legend(override.aes = list(size = 3), ncol = 2, order = 2)
  )

# Display and save autoimmune plot
print(p_autoimmune)
ggsave("plots/fig_autoimmune_shapes.pdf", p_autoimmune, 
       width = width_auto, height = height_auto, units = "in")

In [ ]:
# === INFECTION PLOT ===
width_inf <- 3.6
height_inf <- 5.1

df_infection <- df_plot %>% 
  filter(immune_type == 'infection')

p_infection <- ggplot(df_infection, aes(y = MEruby, x = MEblue)) +
  # Reference lines
  geom_vline(xintercept = 0, linetype = "dotted", color = "grey60") + 
  geom_hline(yintercept = 0, linetype = "dotted", color = "grey60") +
  
  # Main points with shape distinction
  geom_point(aes(color = fine_immune_type, shape = shape_type), 
             size = 3, alpha = 0.8) +
  
  # Col-0 reference point at origin (using data frame method)
  geom_point(data = data.frame(x = 0, y = 0), 
             aes(x = x, y = y), 
             size = 4, shape = 16, color = "#9ccf2d", alpha = 0.9) +
  
  # Ideal plant marker
  geom_point(x = 2.5, y = 0.5, size = 5, shape = 8, color = "black", alpha = 1) +
  
  # Labels
  geom_text_repel(aes(label = source), size = 2, max.overlaps = Inf,
                  box.padding = 0.3, point.padding = 0.2) +
  
  # Col-0 label
  geom_text(data = data.frame(x = 0, y = 0, label = "Col-0"),
            aes(x = x, y = y, label = label),
            size = 2.5, hjust = 1.1, vjust = -0.3, color = "black") +
  
  # Scales and styling
  scale_color_manual(values = custom_colors_grouped, name = "Sample Type") +
  scale_shape_manual(values = shape_mapping, name = "Data Source") +
  
  labs(x = "Growth (MEblue log2FC median)",
       y = "Immunity (MEruby log2FC median)",
       title = "Infection Conditions") +
  
  coord_cartesian(xlim = c(-3.5, 1), ylim = c(-0.8, 3)) +
  
  # Theme
  theme_bw() +
  theme(
    legend.position = "bottom",
    legend.title = element_text(size = 10),
    legend.text = element_text(size = 8),
    axis.text = element_text(size = 8),
    axis.title = element_text(size = 10),
    plot.title = element_text(size = 12, hjust = 0.5)
  ) +
  
  # Legend formatting
  guides(
    color = guide_legend(override.aes = list(size = 3), ncol = 3, order = 1),
    shape = guide_legend(override.aes = list(size = 3), ncol = 2, order = 2)
  )

# Display and save infection plot
print(p_infection)
ggsave("plots/fig_infection_shapes.pdf", p_infection, 
       width = width_inf, height = height_inf, units = "in")


# # Print summary
# cat("Plots created and saved:\n")
# cat("- Autoimmune plot:", nrow(df_autoimmune), "samples\n")
# cat("- Infection plot:", nrow(df_infection), "samples\n")
# cat("- Shape distinction: Circles = Our data, Triangles = Published data\n")
# cat("- Files saved to plots/ directory\n")